In [629]:
import PDF_analyzer as pa
import fitz
import numpy as np
import pandas as pd
import pickle

PATH = "/Users/jedrek/Documents/Studium Volkswirschaftslehre/3. Semester/Long-run dynamics of wealth inequalities/Paper/Data/CBOS pdf"

In [489]:
file = f"{PATH}/CBOS_1_01_1990.pdf"
page = 59
analyzer = pa.PDFAnalyzer(file)
page_sizes = [analyzer.get_page_size(i) for i in range(len(analyzer.doc))]
results = analyzer.get_blocks_on_page(page-1)
size = analyzer.get_page_size(page-1)
drawings = analyzer.get_drawings_on_page(page-1)
analyzer.close()

print(type(results))
print(page_sizes)
print(results)

<class 'list'>
[(595.3200073242188, 841.9199829101562), (595.3200073242188, 841.9199829101562), (595.3200073242188, 841.9199829101562), (595.3200073242188, 841.9199829101562), (595.3200073242188, 841.9199829101562), (612.47998046875, 839.0399780273438), (612.47998046875, 839.0399780273438), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (595.0, 842.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (612.0, 792.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792.0, 612.0), (792

In [490]:
results = np.array(results, dtype=str)
# Get 4th comlumn from results
for i in range(len(results)):
    results[i, 4] = results[i, 4].astype(str).strip()
    results[i, 4] = results[i, 4].replace('\n', ' ')
    
print(results)

[['79.12000274658203' '79.02854919433594' '95.387451171875'
  '88.77854919433594' 'q17' '0' '0']
 ['423.3699951171875' '97.8028335571289' '672.2213745117188'
  '107.5528335571289' 'Wartość Liczebność Procent' '1' '0']
 ['200.6199951171875' '113.5528335571289' '211.4649658203125'
  '123.3028335571289' '97' '2' '0']
 ['200.6199951171875' '129.30282592773438' '211.4649658203125'
  '139.05282592773438' '98' '3' '0']
 ['320.6199951171875' '113.5528335571289' '669.8431396484375'
  '123.3028335571289' 'trudno powiedzieć 15 1,0%' '4' '0']
 ['320.6199951171875' '129.30282592773438' '669.8431396484375'
  '139.05282592773438' 'odmowa odpowiedzi/ brak danych 0 0,0%' '5' '0']
 ['79.12000274658203' '162.27854919433594' '95.387451171875'
  '172.02854919433594' 'q18' '6' '0']
 ['423.3699951171875' '181.05282592773438' '672.2213745117188'
  '190.80282592773438' 'Wartość Liczebność Procent' '7' '0']
 ['80.62000274658203' '196.80282592773438' '234.75926208496094'
  '206.55282592773438' 'Atrybuty standard

In [491]:
vq = np.vectorize(lambda x,y = 'q': x.startswith(y) if isinstance(x, str) else x)
ve = np.vectorize(lambda x: x=='Etykieta')

In [492]:
qs = np.where(vq(results[:,4]))
questions = np.where(ve(results[:,4]))[0]
tables = np.where(vq(results[:,4], 'Wartości z etykietami'))[0]

# Which results[:,4] are integers 
ints = np.vectorize(lambda x: x.isdigit() if isinstance(x, str) else False)(results[:,4])

In [493]:
def find_closest_right_block(ref_idx: int, blocks: np.ndarray) -> int:
                ref_x1 = float(blocks[ref_idx, 2])
                ref_y0 = float(blocks[ref_idx, 1])
                ref_y1 = float(blocks[ref_idx, 3])
                candidates = []
                for i in range(len(blocks)):
                    if i == ref_idx:
                        continue
                    bx0 = float(blocks[i, 0])
                    by0 = float(blocks[i, 1])
                    by1 = float(blocks[i, 3])
                    # Check if block is to the right and vertically overlaps
                    if bx0 >= float(ref_x1 - 1) and (float(min(ref_y1, by1)) - float(max(ref_y0, by0) ) > 0):
                        candidates.append((i, bx0 - ref_x1))
                if not candidates:
                    return -1
                # Return index of the closest block
                candidates.sort(key=lambda x: x[1])
                return candidates[0][0]

In [494]:
def find_closest_right_block(ref_idx: int, blocks: np.ndarray) -> int:
    ref_x0 = float(blocks[ref_idx, 0])
    ref_x1 = float(blocks[ref_idx, 2])
    ref_y0 = float(blocks[ref_idx, 1])
    ref_y1 = float(blocks[ref_idx, 3])
    candidates = []
    for i in range(len(blocks)):
        if i == ref_idx:
            continue
        bx0 = float(blocks[i, 0])
        by0 = float(blocks[i, 1])
        by1 = float(blocks[i, 3])
        # Check if block is to the right and vertically overlaps
        if bx0 > ref_x0 and (float(min(ref_y1, by1)) - float(max(ref_y0, by0) ) > 0):
            candidates.append((i, bx0 - ref_x1))
    if not candidates:
        return -1
    # Return index of the closest block
    candidates.sort(key=lambda x: x[1])
    return candidates[0][0]

In [495]:
print(results[questions[0],1])
a = find_closest_right_block(questions[0], results)


212.55282592773438


In [496]:
results[a,4]

'3. Co, Pana(i) zdaniem, ludzi obecnie najbardziej  oburza i denerwuje? Odp2'

In [497]:
results[np.where(ints),:]

array([[['200.6199951171875', '113.5528335571289', '211.4649658203125',
         '123.3028335571289', '97', '2', '0'],
        ['200.6199951171875', '129.30282592773438', '211.4649658203125',
         '139.05282592773438', '98', '3', '0'],
        ['200.6199951171875', '435.3028259277344', '206.04100036621094',
         '445.0528259277344', '2', '22', '0'],
        ['200.6199951171875', '451.0528259277344', '206.04100036621094',
         '460.8028259277344', '3', '23', '0'],
        ['200.6199951171875', '466.8028259277344', '206.04100036621094',
         '476.5528259277344', '4', '24', '0'],
        ['200.6199951171875', '494.5528259277344', '206.04100036621094',
         '504.3028259277344', '5', '25', '0'],
        ['200.6199951171875', '510.3028259277344', '206.04100036621094',
         '520.0528564453125', '6', '26', '0'],
        ['545.6199951171875', '196.80282592773438', '556.4649658203125',
         '206.55282592773438', '11', '27', '0'],
        ['540.3699951171875', '287.552

In [498]:
results[tables,4][0]

print(tables)
seq_of_rows = []
i=1
while ints[tables+i][0]:
    print(np.mean([float(results[tables+i,1]), float(results[tables+i,3])] ))
    seq_of_rows += [np.mean([float(results[tables+i,1]), float(results[tables+i,3])] )]
    i += 1
    
# Check if its strictly increasing and if not remove the last element until it is
while not all(x<y for x, y in zip(seq_of_rows, seq_of_rows[1:])):
    seq_of_rows = seq_of_rows[:-1]
print(seq_of_rows)


[21]
440.1778259277344
455.9278259277344
471.6778259277344
499.4278259277344
515.1778411865234
201.67782592773438
[440.1778259277344, 455.9278259277344, 471.6778259277344, 499.4278259277344, 515.1778411865234]


/var/folders/y8/4_9g68pj7k136q2yypgp5ysc0000gn/T/ipykernel_13394/3403120540.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(np.mean([float(results[tables+i,1]), float(results[tables+i,3])] ))
/var/folders/y8/4_9g68pj7k136q2yypgp5ysc0000gn/T/ipykernel_13394/3403120540.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  seq_of_rows += [np.mean([float(results[tables+i,1]), float(results[tables+i,3])] )]


In [499]:
results

array([['79.12000274658203', '79.02854919433594', '95.387451171875',
        '88.77854919433594', 'q17', '0', '0'],
       ['423.3699951171875', '97.8028335571289', '672.2213745117188',
        '107.5528335571289', 'Wartość Liczebność Procent', '1', '0'],
       ['200.6199951171875', '113.5528335571289', '211.4649658203125',
        '123.3028335571289', '97', '2', '0'],
       ['200.6199951171875', '129.30282592773438', '211.4649658203125',
        '139.05282592773438', '98', '3', '0'],
       ['320.6199951171875', '113.5528335571289', '669.8431396484375',
        '123.3028335571289', 'trudno powiedzieć 15 1,0%', '4', '0'],
       ['320.6199951171875', '129.30282592773438', '669.8431396484375',
        '139.05282592773438', 'odmowa odpowiedzi/ brak danych 0 0,0%',
        '5', '0'],
       ['79.12000274658203', '162.27854919433594', '95.387451171875',
        '172.02854919433594', 'q18', '6', '0'],
       ['423.3699951171875', '181.05282592773438', '672.2213745117188',
        '190.802

In [500]:
results[:,2].astype(float).max()

720.7990112304688

In [501]:
rect = [results[tables,2][0].astype(float)-5, results[tables,1][0].astype(float),results[:,2].astype(float).max(), max(seq_of_rows)+10]
print(rect)

[201.04100036621094, 407.5528259277344, 720.7990112304688, 525.1778411865234]


In [579]:
f = fitz.open(file)
f_page = f.load_page(page-1)




test = f_page.find_tables(clip = rect, strategy= 'text', snap_tolerance=6)

In [580]:
table_pd = test.tables[0].to_pandas()
display(table_pd)

,Percentyl 75,"25,00",Col2,Col3
0,1,"spadek poziomu życia, pogarszanie się sytuacji",19,"1,3%"
1,,materialnej,,
2,,,,
3,2,"biedy, głodu, nędzy, braku pieniędzy na jedzenie",13,"0,9%"
4,3,"drożyzna, galopada cen, podwyżki",91,"6,1%"
5,4,"kłopoty rynkowe, brak towarów, złe zaopatrzenie,",102,"6,9%"
6,,"kolejki, puste półki",,
7,,,,
8,5,"niskie płace, zarobki, zamrożenie płac",42,"2,8%"
9,6,"niskie renty, emerytury, niskie świadczenia sp...",7,"0,5%"


In [504]:
table_pd = test.tables[0].to_pandas()
# Extract the names of the columns and use them as the new first row
cols = table_pd.columns
table_pd.columns = range(table_pd.shape[1])
table_pd.loc[-1] = cols  # adding a row
table_pd.index = table_pd.index + 1  # shifting index
table_pd = table_pd.sort_index()

In [626]:
def extract_table(page: fitz.Page, ref_idx: int, blocks: np.ndarray) -> pd.DataFrame:
                if blocks[ref_idx+1,4] == "Wartość Liczebność Procent":
                    ref_idx += 1  # move to the next block
                    
                ints = np.vectorize(lambda x: x.isdigit() if isinstance(x, str) else False)(blocks[:,4])
                seq_of_rows = []
                
                i=1
                while ints[ref_idx+i]:
                    seq_of_rows += [np.mean([float(blocks[ref_idx+i,1]), float(blocks[ref_idx+i,3])] )]
                    i += 1
                
                # Check if its strictly increasing and if not remove the last element until it is
                while not all(x<y for x, y in zip(seq_of_rows, seq_of_rows[1:])):
                    seq_of_rows = seq_of_rows[:-1]
                
                if blocks[ref_idx,4] == "Wartość Liczebność Procent":
                    #new_q_idx = np.where(f_vstartswith(blocks[:,4], 'q'))[0][1]
                    rect = [blocks[ref_idx-1,2].astype(float), blocks[ref_idx-1,1].astype(float),
                            blocks[:,2].astype(float).max(), max(seq_of_rows)+10]
                    table = page.find_tables(clip = rect, strategy= 'text', min_words_vertical=1, snap_tolerance=6)
                else: 
                    rect = [blocks[ref_idx,2].astype(float)-5, blocks[ref_idx,1].astype(float),
                            blocks[:,2].astype(float).max(), max(seq_of_rows)+10]
                    table = page.find_tables(clip = rect, strategy= 'text', snap_tolerance=6)
                    
                if blocks[ref_idx,4] == "Wartość Liczebność Procent":
                    print(rect)
                if len(table.tables) == 0 :
                    return pd.DataFrame()
                table = table.tables[0].to_pandas()
                
                return table

In [625]:
#display(results)
extract_table(f_page, qs[0][0], results)

[95.387451171875, 79.02854919433594, 720.7990112304688, 144.17782592773438]


,Col0,Col1,Col2,Col3,Wa,rtość,Liczebn,ość,Pr,ocent
0,97,trudno po,wiedzieć,,,,,15,,"1,0%"
1,98,odmowa,odpowiedzi/,br,ak,danych,,0,,"0,0%"


In [632]:
a = "askdiononsdfina.pdf"
a[:-4]

'askdiononsdfina'

In [647]:
with open("pdf_codebook.json", "rb") as f:
    obj = pickle.load(f)

In [659]:
main = obj['CBOS_1_01_1990']

for df in main:
    display(df[4])
    

""


,Percentyl 75,"3,00",Col2,Col3
0,1,z nadzieją,276,"18,6%"
1,2,"z nadzieją, ale i z obawą",736,"49,5%"
2,3,czy też z obawą,457,"30,8%"
3,7,trudno powiedzieć,2,"0,1%"
4,8,odmowa odpowiedzi/ brak danych,15,"1,0%"


,Percentyl 75,"17,00",Col2,Col3
0,1,"spadek poziomu życia, pogarszanie się sytuacji",161,"10,8%"
1,,materialnej ludzi,,
2,2,"biedy, głodu, nędzy, braku pieniędzy na jedzenie,",226,"15,2%"
3,,utrzymanie rodziny,,
4,,,,
5,3,"drożyzna, galopada cen, podwyżki",342,"23,0%"
6,4,"kłopoty rynkowe, brak towarów, złe zaopatrzenie,",11,"0,7%"
7,,"kolejki, puste półki",,
8,,,,
9,5,"niskie płace, zamrożenie płac",15,"1,0%"


""


""


...

""


""


""


""


""


In [657]:
print(dataframes)

  Percentyl 75                            3,00 Col2   Col3
0            1                      z nadzieją  276  18,6%
1            2       z nadzieją, ale i z obawą  736  49,5%
2            3                 czy też z obawą  457  30,8%
3            7               trudno powiedzieć    2   0,1%
4            8  odmowa odpowiedzi/ brak danych   15   1,0%
